In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import timeit
import matplotlib.pyplot as plt
from preprocess_func import *
%matplotlib inline

#tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
ds = preprocess_filtering_data(date='2011_09_26', out_name='train_2', serieses=[1])

1
(3229, 370, 24, 3)
(3229,)
saved


In [4]:
ds

<TensorDataset shapes: ((3229, 370, 24, 3), (3229,)), types: (tf.float32, tf.int64)>

In [5]:
iter = ds.make_one_shot_iterator()
e = iter.get_next()
#print(features.shape)
with tf.Session() as sess:
    print(sess.run(e)) 
    


(array([[[[0.03529412, 0.03921569, 0.03921569],
         [0.03529412, 0.03529412, 0.03529412],
         [0.03137255, 0.03137255, 0.03529412],
         ...,
         [0.02745098, 0.03529412, 0.05882353],
         [0.03529412, 0.04313726, 0.0627451 ],
         [0.03529412, 0.04313726, 0.09411765]],

        [[0.02745098, 0.03921569, 0.03921569],
         [0.02745098, 0.03529412, 0.03137255],
         [0.02745098, 0.03137255, 0.02745098],
         ...,
         [0.02745098, 0.03137255, 0.05490196],
         [0.03137255, 0.04313726, 0.05882353],
         [0.03137255, 0.05098039, 0.09019608]],

        [[0.02745098, 0.04313726, 0.03529412],
         [0.02745098, 0.03529412, 0.03529412],
         [0.02745098, 0.03137255, 0.03529412],
         ...,
         [0.02745098, 0.03529412, 0.03921569],
         [0.03137255, 0.03529412, 0.03921569],
         [0.03137255, 0.03921569, 0.04705882]],

        ...,

        [[0.5411765 , 0.52156866, 0.5058824 ],
         [0.5294118 , 0.49411765, 0.41568628

In [6]:
iter = ds.make_one_shot_iterator()
features, labels = iter.get_next()

In [ ]:
sess = tf.Session()
iterator = ds.make_initializable_iterator()
next_element = iterator.get_next()
for _ in range(1):
  sess.run(iterator.initializer)
  while True:
    try:
      sess.run(next_element)
    except tf.errors.OutOfRangeError:
      break


In [7]:
# Wrapping all together -> Switch between train and test set
EPOCHS = 10
BATCH_SIZE = 16
# # create a placeholder to dynamically switch between batch sizes
# # batch_size = tf.placeholder(tf.int64)
# # x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
# # dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat()
# # # using two numpy arrays
# # train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
# # test_data = (np.random.sample((20,2)), np.random.sample((20,1)))

# # n_batches = len(train_data[0]) // BATCH_SIZE
# # iter = dataset.make_initializable_iterator()
# # features, labels = iter.get_next()
# # make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)
loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}".format(i, tot_loss / n_batches))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

ValueError: Shapes (3229,) and (3229, 370, 24, 1) are incompatible

In [ ]:
date='2011_09_26'
preprocess_filtering_data(date=date, serieses = [1, 2, 9], dir_path='/home/shahar_zuler/ProjectNexar')

# TO DO:
- split to train and validation in a smarter way

In [ ]:
def get_data(num_training=2000, num_validation=500, num_test=10000, date='2011_09_26'):
    # Load the raw KITTI data
    
    #X_train, y_train =preprocess_filtering_data(date=date, serieses = [1,2], dir_path='/home/shahar_zuler/ProjectNexar')
    #X_test, y_test = preprocess_data(date=date, state = 'Test', dir_path='/home/shahar_zuler/ProjectNexar')
    X_train = np.load('X_train.npy')
    y_train = np.load('y_train.npy')
    print (X_train.shape)
    X_test = X_train
    y_test = y_train
    
    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
#     mask = range(num_test)
#     X_test = X_test[mask]
#     y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
plt.hist(y_train, bins = 47)

In [8]:
# clear old variables
tf.reset_default_graph()

# define our input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 370, 24, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

# define model
def stixel_model(X,y,is_training):
    conv1 = tf.layers.conv2d(inputs=X, filters=64, kernel_size=[11, 5], 
                             padding="same", use_bias=True, activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[8, 4], strides=(8, 4))
    conv2 = tf.layers.conv2d(inputs=pool1, filters=200, kernel_size=[5, 3], 
                             padding="same", use_bias=True, activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[4, 3], strides=(4,3))
    pool2_flat = tf.layers.flatten(inputs=pool2)
    dense3 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout3 = tf.layers.dropout(inputs=dense3, rate=0.5, training=is_training)
    dense4 = tf.layers.dense(inputs=dropout3, units=2048, activation=tf.nn.relu)
    dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=is_training)
    y_out = tf.layers.dense(inputs=dropout4, units=47)
    return y_out

y_out = stixel_model(X,y,is_training)


#predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
 #     "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
  #    "probabilities": tf.nn.softmax(logits, name="softmax_tensor")

In [9]:
# total_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(indices=y, depth=50, axis=-1), logits=y_out)
total_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.one_hot(indices=y, depth=47, axis=-1), logits=y_out)
#tf.summary.histogram('total_loss',total_loss) ####################
mean_loss = tf.reduce_mean(total_loss)




In [10]:
# optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-2, momentum=0.9) 
# tf.train.MomentumOptimizer(learning_rate=1e-2, momentum=0.9)

'''# decay every 10000 steps with a base of 0.5:
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 1e-2
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           decay_steps=10000, decay_rate=0.5, staircase=True)

# Passing global_step to minimize() will increment it at each step.
optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-2, momentum=0.9, decay=0)
train_step = optimizer.minimize(mean_loss, global_step=global_step)'''
# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate #### 5e-4
train_step = optimizer.minimize(mean_loss)

In [ ]:
"""
# batch normalization in tensorflow requires this extra dependency
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)
"""

In [11]:


def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=128, print_every=50,
              training=None, plot_losses=False):
    
    train_writer = tf.summary.FileWriter( './logs/2/train ', sess.graph) #TB
    # have tensorflow compute accuracy
    the_prediction = tf.argmax(predict,axis=1)
    correct_prediction = tf.equal(tf.argmax(predict,axis=1),y)
#     correct_prediction = correct = tf.equal(tf.argmax(tf.nn.sigmoid(predict),axis=1),y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.histogram('accuracy',accuracy) #TB
    tf.summary.scalar('accuracy',accuracy)
    tf.summary.scalar('mean_loss',mean_loss)

    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None

    variables = ['_', mean_loss,correct_prediction, the_prediction ,accuracy] #TB

    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute    
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now}
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            merge = tf.summary.merge_all() #TB
            variables[0] = merge #TB
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
  
            summary, loss, corr,pred, _ = session.run(variables,feed_dict=feed_dict) #TB
            print('pred: ', pred)
            train_writer.add_summary(summary, iter_cnt) #TB
    
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Epoch {0}: Iteration {1}: with minibatch training loss = {2:.3g} and accuracy of {3:.2g}"\
                      .format(e, iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2} Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    
    return total_loss,total_correct

In [12]:
tensors_to_log = {"probabilities": "softmax_tensor"}

saver = tf.train.Saver()
sess = tf.Session()


sess.run(tf.global_variables_initializer())
saver.save(sess, './training_data/stixelnet_model.ckpt')
print('Training')
run_model(session=sess, predict=y_out, loss_val=mean_loss, Xd=features, yd=labels, 
          epochs=20, batch_size=128, print_every=5, training=train_step)




Training


TypeError: unsupported operand type(s) for /: 'Dimension' and 'int'

In [ ]:
print('Validation')
run_model(session=sess, predict=y_out, loss_val=mean_loss, Xd=X_val, yd=y_val, 
          epochs=1, batch_size=128)


In [ ]:
#from preprocessing.preprocess_func_v02_new import *
#preprocess_filtering_data(date='2011_09_26', serieses = [5], dir_path='/home/shahar_zuler/ProjectNexar')

In [ ]:
print('Validation2')
X5 = np.load('X_train_5.npy')
y5 = np.load('y_train_5.npy')
run_model(session=sess, predict=y_out, loss_val=mean_loss, Xd=X5, yd=y5, 
          epochs=1, batch_size=128)